<a href="https://colab.research.google.com/github/tctamedie/EventManagement/blob/master/FraudDetectionCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
fraudTest = pd.read_csv('fraudTest.csv')
fraudTrain = pd.read_csv('fraudTrain.csv')

In [3]:
data= pd.concat([fraudTrain, fraudTest], ignore_index=True)

In [4]:
data.describe(include='object').transpose()

,count,unique,top,freq
trans_date_trans_time,1007752,990621,2020-10-05 19:37:49,4
merchant,1007752,693,fraud_Kilback LLC,3405
category,1007752,14,gas_transport,102527
first,1007752,347,Christopher,20931
last,1007752,477,Smith,22186
gender,1007752,2,F,551836
street,1007751,954,2924 Bobby Trafficway,2485
city,1007751,873,Birmingham,4357
state,1007751,51,TX,73438
job,1007751,486,Film/video editor,7520
